We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [1]:
import duckdb
import pandas as pd

from pathlib import Path

cwd = Path.cwd()

root = None
for p in [cwd] + list(cwd.parents):
    if (p / "db").exists():
        root = p
        break

if root is None:
    raise FileNotFoundError("Could not find a 'db' folder above the current working directory")

DB_PATH = root / "db" / "nflpa.duckdb"
print("Using DB_PATH", DB_PATH)

con = duckdb.connect(str(DB_PATH))

con.execute("PRAGMA threads=4")
con.execute("PRAGMA memory_limit='4GB'")

Using DB_PATH /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb


We verify that the primary panel aligns with the expected regular-season schedule and row counts across all seasons

In [2]:
required = ["pbp", "schedules", "team_week_panel"]
existing = set(con.execute("SHOW TABLES").df()["name"].tolist())

missing = [t for t in required if t not in existing]

print("Missing", missing)
print("OK" if not missing else "STOP, fix missing tables before continuing")

Missing []
OK


We inspect the play-by-play schema to record which optional columns exist allowing subsequent logic to adapt safely without assuming column presence

In [3]:
pbp_cols_df = con.execute("DESCRIBE pbp").df()
pbp_cols = set(pbp_cols_df["column_name"].tolist())

optional_flags = {
    "return_team": "return_team" in pbp_cols,
    "kickoff_type": "kickoff_type" in pbp_cols,
    "onside_kick": "onside_kick" in pbp_cols,
    "safety_kick": "safety_kick" in pbp_cols,
    "free_kick": "free_kick" in pbp_cols,
    "desc": "desc" in pbp_cols,
}

print("Optional columns present", optional_flags)

pbp_cols_df

Optional columns present {'return_team': True, 'kickoff_type': False, 'onside_kick': False, 'safety_kick': False, 'free_kick': False, 'desc': True}


,column_name,column_type,null,key,default,extra
0,play_id,DOUBLE,YES,None,None,None
1,game_id,VARCHAR,YES,None,None,None
2,old_game_id,VARCHAR,YES,None,None,None
3,home_team,VARCHAR,YES,None,None,None
4,away_team,VARCHAR,YES,None,None,None
...,...,...,...,...,...,...
367,xyac_median_yardage,INTEGER,YES,None,None,None
368,xyac_success,DOUBLE,YES,None,None,None
369,xyac_fd,DOUBLE,YES,None,None,None
370,xpass,DOUBLE,YES,None,None,None


We check that team identifiers in the panel align with those in the play-by-play data, ensuring join keys are consistent across datasets to prevent failed matches

In [4]:
panel_cols_df = con.execute("DESCRIBE team_week_panel").df()
panel_cols = set(panel_cols_df["column_name"].tolist())

if "team_id" in panel_cols:
    PANEL_TEAM_COL = "team_id"
elif "team" in panel_cols:
    PANEL_TEAM_COL = "team"
else:
    raise ValueError(f"Could not find a team column in team_week_panel, columns are {sorted(panel_cols)[:50]}")

print("Using panel team column", PANEL_TEAM_COL)

panel_teams = set(
    con.execute(f"SELECT DISTINCT {PANEL_TEAM_COL} AS team_key FROM team_week_panel WHERE {PANEL_TEAM_COL} IS NOT NULL")
      .df()["team_key"]
      .tolist()
)

pbp_posteams = set(
    con.execute("SELECT DISTINCT posteam FROM pbp WHERE posteam IS NOT NULL")
      .df()["posteam"]
      .tolist()
)

intersect = panel_teams.intersection(pbp_posteams)

print("Distinct panel teams", len(panel_teams))
print("Distinct posteam in pbp", len(pbp_posteams))
print("Intersection size", len(intersect))

print("Sample posteam not in panel", sorted(list(pbp_posteams - panel_teams))[:25])
print("Sample panel team not in pbp", sorted(list(panel_teams - pbp_posteams))[:25])


Using panel team column team
Distinct panel teams 35
Distinct posteam in pbp 32
Intersection size 32
Sample posteam not in panel []
Sample panel team not in pbp ['OAK', 'SD', 'STL']


Quick sanity check to confirm that we are not accidentally pulling older schedules seasons that include those codes

In [5]:
con.execute(f"""
SELECT
  {PANEL_TEAM_COL} AS team,
  MIN(season) AS min_season,
  MAX(season) AS max_season,
  COUNT(*) AS rows
FROM team_week_panel
WHERE {PANEL_TEAM_COL} IN ('OAK','SD','STL')
GROUP BY 1
ORDER BY 1
""").df()

,team,min_season,max_season,rows
0,OAK,2012,2019,128
1,SD,2012,2016,80
2,STL,2012,2015,64


We create a minimal special teams base view using safe NULL placeholders for missing optional columns, ensuring downstream SQL remains stable and preventing schema-related breakages

In [6]:
select_return_team = "return_team" if optional_flags["return_team"] else "NULL::VARCHAR AS return_team"
select_kickoff_type = "kickoff_type" if optional_flags["kickoff_type"] else "NULL::VARCHAR AS kickoff_type"
select_onside_kick = "onside_kick" if optional_flags["onside_kick"] else "NULL::INTEGER AS onside_kick"
select_safety_kick = "safety_kick" if optional_flags["safety_kick"] else "NULL::INTEGER AS safety_kick"
select_free_kick = "free_kick" if optional_flags["free_kick"] else "NULL::INTEGER AS free_kick"

select_desc = "\"desc\" AS play_desc" if optional_flags["desc"] else "NULL::VARCHAR AS play_desc"

con.execute(f"""
CREATE OR REPLACE TEMP VIEW pbp_st_base AS
SELECT
  season,
  week,
  game_id,
  posteam,
  defteam,
  {select_return_team},
  CASE
    WHEN play_type IN ('punt','kickoff') THEN COALESCE(return_team, defteam)
    ELSE return_team
  END AS return_team_filled,
  play_type,
  {select_kickoff_type},
  {select_onside_kick},
  {select_safety_kick},
  {select_free_kick},
  {select_desc}
FROM pbp
""")

We filter the play-by-play data to include only regular-season games, ensuring the metric calculations align with the existing panel scope.

In [7]:
con.execute("""
CREATE OR REPLACE TEMP VIEW pbp_st_reg AS
SELECT b.*
FROM pbp_st_base b
JOIN schedules s
  ON b.game_id = s.game_id
WHERE s.game_type = 'REG'
""")

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [8]:
con.execute("""
SELECT
  play_type,
  COUNT(*) AS plays,
  SUM(CASE WHEN return_team IS NULL THEN 1 ELSE 0 END) AS null_return_team,
  AVG(CASE WHEN return_team IS NULL THEN 1.0 ELSE 0.0 END) AS null_return_team_rate
FROM pbp_st_reg
WHERE play_type IN ('punt', 'kickoff')
GROUP BY play_type
ORDER BY play_type
""").df()

,play_type,plays,null_return_team,null_return_team_rate
0,kickoff,34689,658.0,0.018969
1,punt,29463,175.0,0.005940


We build a rare play condition that utilizes explicit kick flags when available and falls back to description keywords when necessary, ensuring robust identification of special teams events across varying data qualities

In [9]:
rare_parts = []

rare_parts.append("kickoff_type IN ('onside', 'safety', 'free_kick', 'free kick', 'safety kick')")
rare_parts.append("onside_kick = 1")
rare_parts.append("safety_kick = 1")
rare_parts.append("free_kick = 1")

if optional_flags["desc"]:
    rare_parts.append("play_desc ILIKE '%onside%'")
    rare_parts.append("play_desc ILIKE '%free kick%'")
    rare_parts.append("play_desc ILIKE '%safety kick%'")
    rare_parts.append("play_desc ILIKE '%fair catch kick%'")
    rare_parts.append("play_desc ILIKE '%free-kick%'")
    rare_parts.append("play_desc ILIKE '%safety-kick%'")

rare_condition = "(" + " OR ".join(rare_parts) + ")"

print("Rare condition SQL")
print(rare_condition)

Rare condition SQL
(kickoff_type IN ('onside', 'safety', 'free_kick', 'free kick', 'safety kick') OR onside_kick = 1 OR safety_kick = 1 OR free_kick = 1 OR play_desc ILIKE '%onside%' OR play_desc ILIKE '%free kick%' OR play_desc ILIKE '%safety kick%' OR play_desc ILIKE '%fair catch kick%' OR play_desc ILIKE '%free-kick%' OR play_desc ILIKE '%safety-kick%')


We create a long events table where each row represents a single special teams event attributed to the correct team for each play component by establishing a granular foundation for team-week aggregation

In [10]:
con.execute(f"""
CREATE OR REPLACE TEMP VIEW st_events_long AS

SELECT
  season,
  week,
  posteam AS team_id,
  'punt' AS component
FROM pbp_st_reg
WHERE play_type = 'punt'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  return_team_filled AS team_id,
  'punt_return' AS component
FROM pbp_st_reg
WHERE play_type = 'punt'
  AND return_team_filled IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'kickoff' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND posteam IS NOT NULL
  AND NOT {rare_condition}

UNION ALL

SELECT
  season,
  week,
  return_team_filled AS team_id,
  'kick_return' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND return_team_filled IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'fg' AS component
FROM pbp_st_reg
WHERE play_type = 'field_goal'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'xp' AS component
FROM pbp_st_reg
WHERE play_type = 'extra_point'
  AND posteam IS NOT NULL

UNION ALL

SELECT
  season,
  week,
  posteam AS team_id,
  'rare' AS component
FROM pbp_st_reg
WHERE play_type = 'kickoff'
  AND posteam IS NOT NULL
  AND {rare_condition}
""")

We aggregate the long events table into the required component columns for each team-week by transforming granular special teams data into a structured feature set ready for panel integration

In [11]:
con.execute("""
CREATE OR REPLACE TEMP VIEW st_components_team_week AS
SELECT
  season,
  week,
  team_id,

  SUM(CASE WHEN component = 'punt' THEN 1 ELSE 0 END) AS ST_Punt_w,
  SUM(CASE WHEN component = 'punt_return' THEN 1 ELSE 0 END) AS ST_PuntReturn_w,
  SUM(CASE WHEN component = 'kickoff' THEN 1 ELSE 0 END) AS ST_Kickoff_w,
  SUM(CASE WHEN component = 'kick_return' THEN 1 ELSE 0 END) AS ST_KickReturn_w,
  SUM(CASE WHEN component = 'fg' THEN 1 ELSE 0 END) AS ST_FG_w,
  SUM(CASE WHEN component = 'xp' THEN 1 ELSE 0 END) AS ST_XP_w,
  SUM(CASE WHEN component = 'rare' THEN 1 ELSE 0 END) AS ST_Rare_w
FROM st_events_long
GROUP BY season, week, team_id
""")

We compute workload totals exactly as defined in the workflow while keeping all component columns for storage in the panel

In [12]:
con.execute("""
CREATE OR REPLACE TEMP VIEW st_load_team_week AS
SELECT
  season,
  week,
  team_id,

  ST_Punt_w,
  ST_PuntReturn_w,
  ST_Kickoff_w,
  ST_KickReturn_w,
  ST_FG_w,
  ST_XP_w,
  ST_Rare_w,

  (ST_Punt_w + ST_PuntReturn_w + ST_Kickoff_w + ST_KickReturn_w + ST_FG_w + ST_XP_w + ST_Rare_w) AS ST_Load_All_w,

  (ST_Kickoff_w + ST_XP_w + ST_FG_w) AS ST_Load_ScoreLinked_w,

  (ST_Punt_w + ST_PuntReturn_w + ST_KickReturn_w + ST_Rare_w) AS ST_Load_NonScore_w
FROM st_components_team_week
""")

Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [13]:
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT season || '-' || week || '-' || team_id) AS distinct_team_weeks,
  MIN(ST_Load_All_w) AS min_all,
  MAX(ST_Load_All_w) AS max_all,
  AVG(ST_Load_All_w) AS avg_all,
  SUM(ST_Rare_w) AS total_rare,
  MAX(ST_Rare_w) AS max_rare
FROM st_load_team_week
""").df()

,rows,distinct_team_weeks,min_all,max_all,avg_all,total_rare,max_rare
0,6782,6782,8.0,31.0,18.219552,855.0,3.0


Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [14]:
con.execute("""
SELECT
  season,
  week,
  SUM(ST_Kickoff_w) AS total_kickoffs_by_kicking_team,
  SUM(ST_KickReturn_w) AS total_kick_returns_by_return_team,
  SUM(ST_Kickoff_w) - SUM(ST_KickReturn_w) AS kickoff_minus_return
FROM st_load_team_week
GROUP BY season, week
ORDER BY ABS(SUM(ST_Kickoff_w) - SUM(ST_KickReturn_w)) DESC
LIMIT 25
""").df()

,season,week,total_kickoffs_by_kicking_team,total_kick_returns_by_return_team,kickoff_minus_return
0,2020,3,0.0,188.0,-188.0
1,2013,15,0.0,186.0,-186.0
2,2020,2,0.0,183.0,-183.0
3,2012,1,0.0,182.0,-182.0
4,2013,14,0.0,179.0,-179.0
5,2020,15,0.0,178.0,-178.0
6,2020,17,0.0,177.0,-177.0
7,2016,16,0.0,176.0,-176.0
8,2019,14,0.0,176.0,-176.0
9,2024,8,0.0,175.0,-175.0


We left join special teams workload components and totals into the team-week panel while preserving all existing rows and filling missing values with zeros

In [15]:
con.execute(f"""
CREATE OR REPLACE TABLE team_week_panel AS
SELECT
  p.*,

  COALESCE(st.ST_Punt_w, 0) AS ST_Punt_w,
  COALESCE(st.ST_PuntReturn_w, 0) AS ST_PuntReturn_w,
  COALESCE(st.ST_Kickoff_w, 0) AS ST_Kickoff_w,
  COALESCE(st.ST_KickReturn_w, 0) AS ST_KickReturn_w,
  COALESCE(st.ST_FG_w, 0) AS ST_FG_w,
  COALESCE(st.ST_XP_w, 0) AS ST_XP_w,
  COALESCE(st.ST_Rare_w, 0) AS ST_Rare_w,

  COALESCE(st.ST_Load_All_w, 0) AS ST_Load_All_w,
  COALESCE(st.ST_Load_ScoreLinked_w, 0) AS ST_Load_ScoreLinked_w,
  COALESCE(st.ST_Load_NonScore_w, 0) AS ST_Load_NonScore_w

FROM team_week_panel p
LEFT JOIN st_load_team_week st
  ON p.season = st.season
 AND p.week = st.week
 AND p.{PANEL_TEAM_COL} = st.team_id
""")

Quick sanity check to confirm row counts are unchanged and verify that workload columns are populated across the full panel

In [16]:
con.execute(f"""
SELECT
  COUNT(*) AS panel_rows,
  COUNT(DISTINCT season || '-' || week || '-' || {PANEL_TEAM_COL}) AS distinct_team_weeks,
  SUM(ST_Load_All_w) AS sum_all,
  SUM(ST_Load_ScoreLinked_w) AS sum_scorelinked,
  SUM(ST_Load_NonScore_w) AS sum_nonscore
FROM team_week_panel
""").df()

,panel_rows,distinct_team_weeks,sum_all,sum_scorelinked,sum_nonscore
0,6782,6782,118415.0,28006.0,90409.0


We compute distribution summaries for each component and total to catch overcounting or misaggregation quickly

In [17]:
con.execute("""
SELECT
  MIN(ST_Punt_w) AS min_punt,
  MAX(ST_Punt_w) AS max_punt,
  AVG(ST_Punt_w) AS avg_punt,

  MIN(ST_PuntReturn_w) AS min_puntret,
  MAX(ST_PuntReturn_w) AS max_puntret,
  AVG(ST_PuntReturn_w) AS avg_puntret,

  MIN(ST_Kickoff_w) AS min_kickoff,
  MAX(ST_Kickoff_w) AS max_kickoff,
  AVG(ST_Kickoff_w) AS avg_kickoff,

  MIN(ST_KickReturn_w) AS min_kickret,
  MAX(ST_KickReturn_w) AS max_kickret,
  AVG(ST_KickReturn_w) AS avg_kickret,

  MIN(ST_FG_w) AS min_fg,
  MAX(ST_FG_w) AS max_fg,
  AVG(ST_FG_w) AS avg_fg,

  MIN(ST_XP_w) AS min_xp,
  MAX(ST_XP_w) AS max_xp,
  AVG(ST_XP_w) AS avg_xp,

  MIN(ST_Rare_w) AS min_rare,
  MAX(ST_Rare_w) AS max_rare,
  AVG(ST_Rare_w) AS avg_rare,

  MIN(ST_Load_All_w) AS min_all,
  MAX(ST_Load_All_w) AS max_all,
  AVG(ST_Load_All_w) AS avg_all
FROM team_week_panel
""").df()

,min_punt,max_punt,avg_punt,min_puntret,max_puntret,avg_puntret,min_kickoff,max_kickoff,avg_kickoff,min_kickret,...,avg_fg,min_xp,max_xp,avg_xp,min_rare,max_rare,avg_rare,min_all,max_all,avg_all
0,0.0,13.0,4.147891,0.0,13.0,4.164996,0.0,0.0,0.0,0.0,...,1.869213,0.0,10.0,2.260248,0.0,3.0,0.119876,0.0,31.0,17.460189


Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [18]:
con.execute(f"""
SELECT
  season,
  week,
  {PANEL_TEAM_COL} AS team,
  ST_Punt_w,
  ST_PuntReturn_w,
  ST_Kickoff_w,
  ST_KickReturn_w,
  ST_FG_w,
  ST_XP_w,
  ST_Rare_w,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
ORDER BY ST_Load_All_w DESC, season DESC, week DESC, {PANEL_TEAM_COL}
LIMIT 50
""").df()

,season,week,team,ST_Punt_w,ST_PuntReturn_w,ST_Kickoff_w,ST_KickReturn_w,ST_FG_w,ST_XP_w,ST_Rare_w,ST_Load_All_w,ST_Load_ScoreLinked_w,ST_Load_NonScore_w
0,2023,8,NYG,13.0,11.0,0.0,3.0,3.0,1.0,0.0,31.0,4.0,27.0
1,2013,7,IND,9.0,7.0,0.0,7.0,3.0,4.0,1.0,31.0,7.0,24.0
2,2023,8,NYJ,11.0,13.0,0.0,3.0,2.0,1.0,0.0,30.0,3.0,27.0
3,2013,11,HOU,9.0,11.0,0.0,5.0,3.0,2.0,0.0,30.0,5.0,25.0
4,2013,7,DEN,7.0,9.0,0.0,9.0,2.0,3.0,0.0,30.0,5.0,25.0
5,2013,1,BAL,10.0,7.0,0.0,8.0,2.0,3.0,0.0,30.0,5.0,25.0
6,2013,1,DEN,7.0,10.0,0.0,6.0,0.0,7.0,0.0,30.0,7.0,23.0
7,2018,7,MIN,8.0,8.0,0.0,4.0,4.0,4.0,1.0,29.0,8.0,21.0
8,2017,6,CHI,9.0,7.0,0.0,7.0,2.0,3.0,1.0,29.0,5.0,24.0
9,2017,6,DET,5.0,7.0,0.0,10.0,2.0,5.0,0.0,29.0,7.0,22.0


Quick sanity check to confirm that return team fields are populated often enough for punts and kickoffs and to quantify how many return events may be missing

In [19]:
con.execute("""
SELECT
  season,
  week,
  team,
  ST_Load_All_w,
  ST_Load_ScoreLinked_w,
  ST_Load_NonScore_w
FROM team_week_panel
ORDER BY season DESC, week DESC, team
LIMIT 30
""").df()

,season,week,team,ST_Load_All_w,ST_Load_ScoreLinked_w,ST_Load_NonScore_w
0,2024,18,ARI,18.0,7.0,11.0
1,2024,18,ATL,17.0,7.0,10.0
2,2024,18,BAL,17.0,5.0,12.0
3,2024,18,BUF,17.0,3.0,14.0
4,2024,18,CAR,17.0,6.0,11.0
5,2024,18,CHI,19.0,4.0,15.0
6,2024,18,CIN,18.0,5.0,13.0
7,2024,18,CLE,17.0,2.0,15.0
8,2024,18,DAL,19.0,5.0,14.0
9,2024,18,DEN,14.0,6.0,8.0
